In [4]:
# https://github.com/dadejecova/Natural-Language-Python/blob/main/dataset/trainset.csv
# We're going to install those libraries
# !pip install transformers
# !pip install sentencepiece
# !pip install rouge-score

# GPUT4 is the environment that we'll use

In [5]:
import transformers

transformers.__version__

'4.52.2'

In [6]:
import torch

import pandas as pd

from transformers import T5Tokenizer, T5ForConditionalGeneration

from transformers.optimization import Adafactor

from rouge_score import rouge_scorer

In [11]:
import io
import requests
import pandas as pd

In [16]:

url = 'https://raw.githubusercontent.com/dadejecova/Natural-Language-Python/main/dataset/trainset.csv'
response = requests.get(url)

In [18]:
if response.status_code == 200:
  #we read this with pnds
  restaurant_df = pd.read_csv(io.StringIO(response.text))
  print(restaurant_df.sample(10))
else:
  print("Failed to fetch Datset")

                                                      mr                                                ref
16186  name[Aromi], eatType[coffee shop], food[Indian...  An adult coffee shop called Aromi serves India...
14916  name[Zizzi], eatType[coffee shop], priceRange[...  At riverside the good rated coffee shop Zizzi ...
38490  name[Blue Spice], eatType[coffee shop], priceR...  Blue Spice is a children friendly coffee shop ...
10786  name[Browns Cambridge], eatType[coffee shop], ...  Want some fast food and a family friendly atti...
39424  name[The Phoenix], food[Chinese], priceRange[c...  The Phoenix has received high customer ratings...
16504  name[The Rice Boat], food[Japanese], customer ...  The Rice Boat is not family-friendly that sell...
25006  name[Cocum], eatType[coffee shop], food[French...  Cocum is a family friendly coffee shop that se...
37516  name[Wildwood], eatType[pub], food[Italian], p...  An Italian pub named Wildwood with high custom...
1268   name[Fitzbillies], ea

In [19]:
restaurant_df.shape

(42061, 2)

In [20]:
restaurant_df['mr'][3]

'name[The Mill], eatType[coffee shop], food[French], priceRange[£20-25], area[riverside], near[The Sorrento]'

In [21]:
restaurant_df['ref'][3]

'Located near The Sorrento is a French Theme eatery and coffee shop called The Mill, with a price range at £20-£25 it is in the riverside area.'

In [24]:
restaurant_df_sampled = restaurant_df.sample(frac = 0.3)

restaurant_df_sampled.shape

(12618, 2)

In [25]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device} device")

Using cuda device


In [26]:
tokenizer = T5Tokenizer.from_pretrained("t5-small", legacy=False)

tokenizer

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

loading file spiece.model from cache at /root/.cache/huggingface/hub/models--t5-small/snapshots/df1b051c49625cf57a3d0d8d3863ed4d13564fe4/spiece.model
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--t5-small/snapshots/df1b051c49625cf57a3d0d8d3863ed4d13564fe4/tokenizer_config.json
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--t5-small/snapshots/df1b051c49625cf57a3d0d8d3863ed4d13564fe4/tokenizer.json
loading file chat_template.jinja from cache at None
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


T5Tokenizer(name_or_path='t5-small', vocab_size=32000, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<extra_id_4

In [30]:
tokens = tokenizer.tokenize("very budget-friendly restaurant")
tokens

['▁very', '▁budget', '-', 'friendly', '▁restaurant']

In [31]:
ids = tokenizer.convert_tokens_to_ids(tokens)
ids

[182, 1487, 18, 4905, 2062]